# Imports

In [1]:
import pandas as pd
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
# this adds the src folder in the sys path, where the metric_utils.py file is
# not needed if this notebook is in the same folder, but uncomment to access from the data subfolders
sys.path.append( '..' )
from metric_utils import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Result path

In [2]:
dataset_name = 'jigsaw'
model_name = 'bert-base-uncased'
run_name = "run 1"

# change to f'../../results/{dataset_name}' when using inside one of the data subfolders
run_folder = f'../../results/{dataset_name}/{run_name}'
test_csv_filepath = os.path.join(f'../../results/{dataset_name}', 'test.csv')

model_folder = os.path.join(run_folder, model_name) # for this particular model
result_folder = os.path.join(model_folder, 'normal')

result_filepath = os.path.join(result_folder, 'results.csv')

# Dataset
The dataset can be preprocessed from the original dataset to be used here. To simplify things, I saved the preprocessed datasets during the tokenizing process as csv files and then downloaded them in the corresponding dataset folder of [`result`](../results/) directory.

You can recreate the processed datasets using the tokenize notebooks for that particular dataset. That would give you train, test and validation csv files as well as the tokenized data.

In [6]:
result = pd.read_csv(result_filepath)
test_df = pd.read_csv(test_csv_filepath)
test_df.head(2)

,id,comment_text,labels,male,female,transgender,white,black,asian,latino,heterosexual,homosexual_gay_or_lesbian,bisexual,christian,jewish,muslim,hindu
0,7084460,"""while arresting a man for resisting arrest"".\n\nIf you cop-suckers can't see a problem with this, then go suck the barrel of a Glock.",True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7141509,"NO ! There are no alternative facts. Go check for yourself. It is people like you, who have no idea what you are talking about that has gotten this State and Country into the mess it is in. People who think the Goverment, be it State or Federal, can spend the peoples money better than they can, is stupid and nonsensical. Politicians use taxes as Personal slush accounts to continue their carrers, buying votes from the lame and the lazy.",True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Doing this reduces the test file size, makes things easier with git
if 'comment_text' in test_df.columns:
    test_df.drop(columns='comment_text', inplace=True)
    test_df.to_csv(test_csv_filepath, index=False)

test_df.fillna(0, inplace=True)

In [8]:
identities = ['male', 'female', 'transgender', 'white', 'black', 'asian']
for identity in identities:
    print(identity, test_df[test_df[identity]>=0.5][target_column].value_counts())

male False    3716
True      670
Name: labels, dtype: int64
female False    4463
True      692
Name: labels, dtype: int64
transgender False    209
True      51
Name: labels, dtype: int64
white False    1710
True      742
Name: labels, dtype: int64
black False    1016
True      503
Name: labels, dtype: int64
asian False    408
True      46
Name: labels, dtype: int64
latino False    174
True      51
Name: labels, dtype: int64
heterosexual False    112
True      29
Name: labels, dtype: int64
homosexual_gay_or_lesbian False    775
True     290
Name: labels, dtype: int64
bisexual False    27
True      7
Name: labels, dtype: int64
christian False    3809
True      417
Name: labels, dtype: int64
jewish False    697
True     138
Name: labels, dtype: int64
muslim False    1557
True      483
Name: labels, dtype: int64
hindu False    46
True      5
Name: labels, dtype: int64


## Split test result

In [ ]:
# only calculate test result
result = result[result['split']=='test']

# drop split column
result.drop(columns=['split'], inplace=True)

## Merge

In [ ]:
extra_columns = [col for col in test_df.columns if col in result.columns and col!=id_column]
result = result.merge(test_df.drop(columns=extra_columns), on=id_column, how='inner').reset_index(drop=True)

# Evaluation

## Convert probability to prediction

In [ ]:
result[prediction_column] = result[probability_column] >=0.5

## Identity groups

In [ ]:
group_map = {
    'gender': {
        'unprivileged':['female'],
        'privileged':['male']
    },
    'race': {
        'unprivileged':['black'],
        'privileged': ['white']
    }
}

identities = []
for group_key in group_map.keys():
    subgroup_map = group_map[group_key]
    for subgroup_key in subgroup_map.keys():
        identities.extend(subgroup_map[subgroup_key])

print(identities)

### Binarize identity and target columns

In [ ]:
result = binarize(result, [target_column] + identities)

## Bias

In [ ]:
bias_results = get_all_bias(group_map, result)
bias_results

In [ ]:
bias_results.round(3).to_csv(os.path.join(model_folder, 'bias.csv'), index=False)

## Overall metrics

In [ ]:
overall_results = get_overall_results(group_map, result)
overall_results

In [ ]:
overall_results.round(3).to_csv(os.path.join(model_folder, 'overall_results.csv'), index=False)

## Overall bias auc

In [ ]:
overall_auc = overall_results[overall_results['metrics']=='auc']['Total'].values[0]

In [ ]:
temp = overall_results.set_index('metrics').T
privileged, unprivileged = [], []
for group_key in group_map.keys():
    privileged.extend(group_map[group_key]['privileged'])
    unprivileged.extend(group_map[group_key]['unprivileged'])

privileged_bias_auc = get_final_metric(temp[temp.index.isin(privileged)], overall_auc)
unprivileged_bias_auc = get_final_metric(temp[temp.index.isin(unprivileged)], overall_auc)

print(privileged_bias_auc, unprivileged_bias_auc)

# Count
The the `train.csv` file from `experiment/run` folders for this corresponding run. And manually copy it in the path run_folder points to.

In [ ]:
train = pd.read_csv(os.path.join(run_folder, 'train.csv'))

In [ ]:
count_dict = {
    'Identity':identities,
    '0 (train)':[],
    '1 (train)':[],
    '0 (test)':[],
    '1 (test)':[],
}
for identity in identities:
    train_neg, train_pos = train[train[identity]>=0.5][target_column].value_counts().to_numpy()
    test_neg, test_pos = test_df[test_df[identity]>=0.5][target_column].value_counts().to_numpy()
    count_dict['0 (train)'].append(train_neg)
    count_dict['1 (train)'].append(train_pos)
    count_dict['0 (test)'].append(test_neg)
    count_dict['1 (test)'].append(test_pos)

count_df = pd.DataFrame(count_dict)
print(count_df)

In [ ]:
count_df.to_csv(os.path.join(run_folder, 'count.csv'), index=False)